In [1]:
#性能测量
def meansure_performance(y_predict_lis):
    TP=0
    FP=0
    TN=0
    FN=0
    #测试样本数
    sample_num=x_test.shape[0]
    for i in range(sample_num):
        y_predict=y_predict_lis[i]
        if y_predict==1 and y_test[i]==1:
            TP+=1
        elif y_predict==-1 and y_test[i]==-1:
            TN+=1
        elif y_predict==1 and y_test[i]==-1:
    #false poistive
            FP+=1
        elif y_predict==-1 and y_test[i]==1:
    #false negative
            FN+=1 
    accuracy=(TP+TN)/(TP+FP+TN+FN)
    recall=TP/(TP+FN)
    specificity=TN/(TN+FP)
    precision=TP/(TP+FP)
    F_measuer=(2*precision*recall)/(precision+recall)
    return accuracy,recall,specificity,precision,F_measuer

In [2]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from skfeature.function.statistical_based import CFS
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_jinhuijiu.csv')
data_rfe_array=np.array(data_rfe)
fea=data_rfe_array[:,:49]
target=data_rfe_array[:,49]
x_test=fea[:170,:]
y_test=target[:170]
x_train=fea[170:,:]
y_train=target[170:]

In [3]:
techini_indicator=data_rfe.columns.tolist()[:-1]

# SVM

In [57]:
from sklearn.svm import SVC
estima_SVM=SVC(kernel='linear',C=0.0039)
estima_SVM.fit(x_train,y_train)
SVM_result=estima_SVM.predict(x_test)
performan_SVM=meansure_performance(SVM_result)
print(performan_SVM)

(0.8294117647058824, 0.8157894736842105, 0.8404255319148937, 0.8051948051948052, 0.8104575163398693)


 # CFS-SVM

In [58]:

from skfeature.function.statistical_based import CFS
CFS_selected_feature = CFS.cfs(x_train,y_train)
CFS_fea_sele=list(CFS_selected_feature)
CFS_fea_sele

[8, 31, 28, 1, 2, 3]

In [59]:
np.array(techini_indicator)[CFS_fea_sele],len(np.array(techini_indicator)[CFS_fea_sele])

(array(['VR', 'VROC', 'J', 'TAPI', 'VMA', 'VMACD'], dtype='<U8'), 6)

In [60]:
estimator_SVM_CFS=SVC(kernel='linear',C=0.0039)
estimator_SVM_CFS.fit(x_train[:,CFS_fea_sele],y_train)
CFS_SVM_result=estimator_SVM_CFS.predict(x_test[:,CFS_fea_sele])

In [61]:
performan_CFS_SVM=meansure_performance(CFS_SVM_result)
print(performan_CFS_SVM)

(0.5647058823529412, 0.3815789473684211, 0.7127659574468085, 0.5178571428571429, 0.4393939393939394)


# sequencial-backward-elimination SVM

In [62]:

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class back_SVM:
    def __init__(self,X_train,Y_train,c):
        self.X_train=X_train
        self.Y_train=Y_train
        self.C=c
    def SBE(self,folds):
        fea_num=self.X_train.shape[1]
        candi_fea=[i for i in range(fea_num)]
        best_subset=[i for i in range(fea_num)]
        model=SVC(kernel='linear',C=self.C)
        Best_accuracy=mean(cross_val_score(model,self.X_train,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1))
        Success=True
        while Success:
            Success=False
            for f in candi_fea:
                candi_fea_copy=candi_fea.copy()
                candi_fea_copy.remove(f)
                x_sele=self.X_train[:,candi_fea_copy]
                n_scores=cross_val_score(model,x_sele,self.Y_train,scoring="accuracy",cv=folds,n_jobs=-1)
                acc=mean(n_scores)
                if acc>=Best_accuracy:
                    #print(f)
                    best_subset=candi_fea_copy
                    Success=True
                    Best_accuracy=acc
            print(1)
            if Success==True:
                candi_fea=best_subset
            print(len(candi_fea))
        return best_subset
                
        

In [63]:
estimator_SVM_back=back_SVM(x_train,y_train,0.0039)
candi_fea_SBE=estimator_SVM_back.SBE(5)

1
48
1
47
1
46
1
45
1
44
1
43
1
42
1
42


In [64]:
np.array(techini_indicator)[candi_fea_SBE],len(np.array(techini_indicator)[candi_fea_SBE])

(array(['VMA', 'VMACD', 'VOSC', 'VSTD', 'CR', 'PSY', 'VR', 'AR', 'BR',
        'WAD', 'MOBV', 'PVT', 'WVAD', 'BBI', 'EXPMA', 'MACD', 'MTM',
        'PRICEOSC', 'BIAS', 'CCI', 'DBCD', 'DPO', 'K', 'J', 'RSI', 'SRDM',
        'VROC', 'VRSI', 'WR', 'ADTM', 'CDP', 'DPTB', 'JDQS', 'JDRS', 'MI',
        'MICD', 'RCCD', 'SRMI', 'ATR', 'CVLT', 'STD', 'VHF'], dtype='<U8'),
 42)

In [65]:
candi_fea_SBE

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 17,
 18,
 19,
 20,
 22,
 23,
 24,
 25,
 26,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 42,
 43,
 44,
 45,
 47,
 48]

In [66]:
np.array(techini_indicator)[16],np.array(techini_indicator)[21],np.array(techini_indicator)[27],np.array(techini_indicator)[41]

('MA', 'TRIX', 'D', 'RC')

In [88]:
np.array(techini_indicator)[46]

'MASS'

In [68]:
estimator_SVM_SBE=SVC(kernel='linear',C=0.0039)
estimator_SVM_SBE.fit(x_train[:,candi_fea_SBE],y_train)
SBE_SVM_result=estimator_SVM_SBE.predict(x_test[:,candi_fea_SBE])

In [69]:
performan_SBE_SVM=meansure_performance(SBE_SVM_result)
print(performan_SBE_SVM)

(0.8235294117647058, 0.8026315789473685, 0.8404255319148937, 0.8026315789473685, 0.8026315789473685)


# reliefF-svm 

In [70]:
from skfeature.function.similarity_based import reliefF
relief_fea_score=reliefF.reliefF(x_train,y_train)
relief_candi_fea=list(reliefF.feature_ranking(relief_fea_score))[:13]

In [71]:
np.array(techini_indicator)[relief_candi_fea],len(np.array(techini_indicator)[relief_candi_fea])

(array(['MOBV', 'VRSI', 'QRR', 'RSI', 'WR', 'DPO', 'BIAS', 'VROC', 'PVT',
        'TAPI', 'WAD', 'VR', 'PSY'], dtype='<U8'),
 13)

In [72]:
estimator_SVM_relief=SVC(kernel='linear',C=0.0039)
estimator_SVM_relief.fit(x_train[:,relief_candi_fea],y_train)
relief_SVM_result=estimator_SVM_relief.predict(x_test[:,relief_candi_fea])

In [73]:
performan_relief_SVM=meansure_performance(relief_SVM_result)
print(performan_relief_SVM)

(0.8235294117647058, 0.7894736842105263, 0.851063829787234, 0.8108108108108109, 0.8)


# RFE-SVM

In [74]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
   
    def featu_subset(self,featur_num):
        kf=KFold(n_splits=5,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            if len(candi_fea)==featur_num:
                break
        return candi_fea

In [75]:
rfe_SVM_model=rfe_SVM(x_train,y_train,0.0039)
rfe_SVM_sele_featu_subset=rfe_SVM_model.featu_subset(13)

In [76]:
np.array(techini_indicator)[rfe_SVM_sele_featu_subset],len(np.array(techini_indicator)[rfe_SVM_sele_featu_subset])

(array(['QRR', 'PSY', 'AR', 'MOBV', 'MACD', 'DBCD', 'K', 'D', 'RSI', 'WR',
        'ADTM', 'JDQS', 'JDRS'], dtype='<U8'),
 13)

In [77]:
estimator_SVM_rfe=SVC(kernel='linear',C=0.0039)
estimator_SVM_rfe.fit(x_train[:,rfe_SVM_sele_featu_subset],y_train)
rfe_SVM_result=estimator_SVM_rfe.predict(x_test[:,rfe_SVM_sele_featu_subset])

In [78]:
performan_rfe_SVM=meansure_performance(rfe_SVM_result)
print(performan_rfe_SVM)

(0.8470588235294118, 0.7894736842105263, 0.8936170212765957, 0.8571428571428571, 0.8219178082191781)


# SVM-RFE-RELIEF

In [8]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,alpha,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            #print(rfe_score)
            total_score=alpha*relief_score+(1-alpha)*rfe_score
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            total_score=alpha*relief_score+(1-alpha)*rfe_score
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            if len(candi_fea)==optim_featur_num:
                return candi_fea

In [80]:
# k=4# k=5, 0.8
rfe_relief_SVM_model=rfe_relief_SVM(x_train,y_train,0.0039)
rfe_relief_SVM_sele_featu_subset=rfe_relief_SVM_model.sele_feature(0.35,13)

In [81]:
np.array(techini_indicator)[rfe_relief_SVM_sele_featu_subset],len(np.array(techini_indicator)[rfe_relief_SVM_sele_featu_subset])

(array(['QRR', 'PSY', 'MOBV', 'MACD', 'BIAS', 'K', 'D', 'RSI', 'VRSI',
        'WR', 'ADTM', 'CDP', 'JDQS'], dtype='<U8'),
 13)

In [82]:
estimator_SVM_rfe_reliefF=SVC(kernel='linear',C=0.0039)
estimator_SVM_rfe_reliefF.fit(x_train[:,rfe_relief_SVM_sele_featu_subset],y_train)
rfe_reliefF_SVM_result=estimator_SVM_rfe_reliefF.predict(x_test[:,rfe_relief_SVM_sele_featu_subset])
performan_rfe_reliefF_SVM=meansure_performance(rfe_reliefF_SVM_result)
print(performan_rfe_reliefF_SVM)

(0.8470588235294118, 0.8026315789473685, 0.8829787234042553, 0.8472222222222222, 0.8243243243243243)
